# 0. Python and OpenCV version

In [ ]:
import cv2
from platform import python_version
print("Python version : " + python_version())
print("cv2 version : " + cv2.__version__)
# Python version : 3.8.2
# cv2 version : 4.4.0

# 1. Extract Information about a video

In [ ]:
import cv2

# write a vide path which you want to get information
video_path = "/Users/okumurayuki/Desktop/videos_0920/19500Hz.mp4"
cap = cv2.VideoCapture(video_path)
width = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
count = cap.get(cv2.CAP_PROP_FRAME_COUNT)
fps = cap.get(cv2.CAP_PROP_FPS)

print("width:{}, height:{}, count:{}, fps:{}".format(width,height,count,fps))

# 2. Preprocessing (divide each video frame to an image and save)   

In [ ]:
import os
import cv2
from glob import glob

#write path where your videos locate
dir_name = "/Users/okumurayuki/Desktop/videos_0920"

for video_path in glob(dir_name + '/*.mp4'):

    cap = cv2.VideoCapture(video_path)
    
    #make a folder in the same folder with  its video name 
    #video_path[:-4]  is a folder name where devided frames are saved
    folder_path = video_path[:-4]
    if not os.path.isdir(folder_path):
            os.makedirs(folder_path)

    num = 0
    while(cap.isOpened()):
        ret, frame = cap.read()
        if ret == True:
            # save image from frame000
            cv2.imwrite(folder_path +"/frame{:0=3}".format(num)+".jpg",frame)       
            num += 1
        else:
            break

    cap.release()

# 3. Evaluate by caluculating difference between base image and each frame

In [ ]:
import cv2
import os
import matplotlib.pyplot as plt
import numpy as np
from glob import glob
import pandas as pd 

#write a folder path where videos locate
dir_name = "/Users/okumurayuki/Desktop/videos_0920"

for folder_path in glob(dir_name + '/**/'):
    video_path = folder_path[:-1] + '.mp4'
    cap = cv2.VideoCapture(video_path)
    count = cap.get(cv2.CAP_PROP_FRAME_COUNT)
    
    #define list for each evaluation method
    Diff_frame = []
    
    for i in range(int(count)):
        image_path = folder_path + "frame{:0=3}".format(i)+".jpg"
        image = cv2.imread(image_path)
        gray_image = cv2.cvtColor(image,cv2.COLOR_RGB2GRAY)
        if i == 0:
            base_image = gray_image
        #calculate abstract difference base image and each image
        diff_frame = cv2.absdiff(gray_image, base_image)
        #add sum of diff_frame to Diff_frame
        Diff_frame.append(diff_frame.sum())
      
    #show result with graph
    fig = plt.figure() 
    x = list(range(int(count)))
    y = Diff_frame

    #define figure tittle which is same with the video title
    folder_path = folder_path[:-1]

    plt.plot(x,y)
    plt.grid(True) 
    plt.xlabel("Video frame")
    plt.ylabel("Sum")
    plt.xlim(x[0], x[-1])
    plt.title(os.path.basename(folder_path))
    plt.show()
    
    # save result as jpg and csv
    save_folder = dir_name + "/Disturbance/"
    if not os.path.isdir(save_folder):
        os.makedirs(save_folder )

    df = pd.DataFrame({'frame': x, 'Sum': y})
    blankIndex = [''] * len(df)
    df.index = blankIndex
    df.to_csv(save_folder + os.path.basename(folder_path) + 'Hz.csv')
    fig.savefig(save_folder + os.path.basename(folder_path) +'Hz.jpg') 

# 4.1 make a summary of 3 by caluculating mean after 100th frame

In [ ]:
import cv2
import os
import matplotlib.pyplot as plt
import numpy as np
from glob import glob
import pandas as pd 

#write a folder path where videos locate
dir_name = "/Users/okumurayuki/Desktop/videos_0920"

#define lists for summary
Frequency = []
Diff_mean = []

for folder_path in glob(dir_name + '/Disturbance/' + '/*.csv'):
    
    df = pd.read_csv(folder_path)
    #define lists for caluculating mean of laplaican, sobel and FFT
    diff = []

    #delete first 100 frame results
    for i in range(len(df['Sum'])):
        if i >= 100:
            diff.append(df['Sum'][i])

    Frequency.append(int(os.path.basename(folder_path[:-6])))
    Diff_mean.append(sum(diff)/len(diff))

#sort because the list of Frequency is not ordered 
Frequency_sorted = sorted(Frequency)
Diff_sorted = []

for i in range(len(Frequency_sorted)):
    for j in range(len(Frequency_sorted)):
        if Frequency_sorted[i] == Frequency[j]:
            Diff_sorted.append(Diff_mean[j])
            
#show result with graph          
fig = plt.figure()
    
#define figure tittle which is same with the video title
folder_path = folder_path[:-1]
x = Frequency_sorted
y = Diff_sorted 

plt.plot(x,y)
plt.grid(True) 
plt.xlabel("Frequency [Hz]")
plt.ylabel("Mean")
plt.xlim(x[0], x[-1])
plt.title('Summary')
plt.show()

# save result as jpg and csv
save_folder = dir_name + "/Disturbance/Summary/"
if not os.path.isdir(save_folder):
    os.makedirs(save_folder )

df = pd.DataFrame({'frame': x, 'Mean': y})
blankIndex = [''] * len(df)
df.index = blankIndex
df.to_csv(save_folder + 'Sumarry_mean.csv')
fig.savefig(save_folder + 'Summary_mean.jpg') 


# 4.2 make a summary of 3 by caluculating max after 100th frame

In [ ]:
import cv2
import os
import matplotlib.pyplot as plt
import numpy as np
from glob import glob
import pandas as pd 

#write a folder path where videos locate
dir_name = "/Users/okumurayuki/Desktop/videos_0920"

#define lists for summary
Frequency = []
Diff_max = []

for folder_path in glob(dir_name + '/Disturbance/' + '/*.csv'):
    df = pd.read_csv(folder_path)
    #define lists for caluculating mean of laplaican, sobel and FFT
    diff = []

    #delete first 100 frame results
    for i in range(len(df['Sum'])):
        if i >= 100:
            diff.append(df['Sum'][i])

    Frequency.append(int(os.path.basename(folder_path[:-6])))
    Diff_max.append(max(diff))

#sort because the list of Frequency is not ordered 
Frequency_sorted = sorted(Frequency)
Diff_sorted = []

for i in range(len(Frequency_sorted)):
    for j in range(len(Frequency_sorted)):
        if Frequency_sorted[i] == Frequency[j]:
            Diff_sorted.append(Diff_max[j])      

#show result with graph
fig = plt.figure()
    
#define figure tittle which is same with the video title
folder_path = folder_path[:-1]
x = Frequency_sorted
y = Diff_sorted 

plt.plot(x,y)
plt.grid(True) 
plt.xlabel("Frequency [Hz]")
plt.ylabel("Max")
plt.xlim(x[0], x[-1])
plt.title('Summary')
plt.show()

# save result as jpg and csv    
save_folder = dir_name + "/Disturbance/Summary/"
if not os.path.isdir(save_folder):
    os.makedirs(save_folder )
df = pd.DataFrame({'frame': x, 'Max': y})
blankIndex = [''] * len(df)
df.index = blankIndex
df.to_csv(save_folder + 'Sumarry_max.csv')
fig.savefig(save_folder + 'Summary_max.jpg') 